In [ ]:
from urllib.request import urlopen
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time

## Configuration data

In [ ]:
startpage = "https://www2.proz.com/find?lpe=3&st=1&q=&sl=esl&tl=eng&nit=y&nl=eng&lpe=3&ee=specialty&v=h&m=n"
# "https://www2.proz.com/find?lpe=3&st=1&q=&sl=deu&tl=eng&nit=y&nl=eng&lpe=3&ee=specialty&v=h"
# "https://www2.proz.com/find?slwz=esl&tlwz=eng&lpe=3&nl_wz=target&st=1&q=&sl=esl&tl=eng&nit=y&lpe=8&ee=specialty"
webprefix = "https://www2.proz.com"
username = "m.marti.menzel@eurotransmit.com"
password = "CuRaBRp9g8buXDr"

## Utility functions

In [ ]:
def grabCV(trltrPage, trltrtName):
    trltrHref = ""
    dlFileName = ""
    
    r = requests.get(trltrPage, cookies=s.cookies)
    bs = BeautifulSoup(r.content, 'html.parser')

    # Find the tag that contains "CV/Resume"
#   resumeTag = bs.find(lambda tag: len(tag.findAll()) == 0 and "CV/Resume" in tag.text) # Alternative statement
    resumeText = bs.body.find(text=re.compile('CV/Resume'))
    
    # If the tag is hidden, resort to Selenium
    if (resumeText == None):
        selContents = selActionVisible(trltrPage)
        if selContents == None:
            return None
        bsS = BeautifulSoup(selContents, 'html.parser')
        resumeText = bsS.body.find(text=re.compile('CV/Resume'))
        if (resumeText == None):
            return None

    resumeTag = resumeText.parent

    resumeTagA = resumeTag.nextSibling.nextSibling.find('a')  # Double nextSibling because of blank spaces       

    if (resumeTagA is not None):
        trltrHref = resumeTagA.attrs['href']
    else:
        return None

    print(trltrHref)
    # Open the stream to the local file
    if (trltrHref != ""):
        rd = requests.get(trltrHref)
        print(rd.headers["Content-Type"])
        if (rd.headers["Content-Type"] == 'application/msword'):
            dlFileName = trltrtName + ".doc"
        elif (rd.headers["Content-Type"] == 'application/pdf'):
            dlFileName = trltrtName + ".pdf"
        elif (rd.headers["Content-Type"] == 'application/rtf'):
            dlFileName = trltrtName + ".rtf"
        elif (rd.headers["Content-Type"] == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'):    
            dlFileName = trltrtName + ".docx"
        elif (rd.headers["Content-Type"] == 'application/vnd.oasis.opendocument.text'):    
            dlFileName = trltrtName + ".odt"
        else:
            dlFileName = trltrtName + ".txt"
        with open(dlFileName, 'wb') as outfile:
            outfile.write(rd.content)
    
        

In [ ]:
def selActionVisible(trltrPage):
    driver.get(trltrPage)
    for setCookie in cookieList:
        driver.add_cookie(setCookie)

    driver.implicitly_wait(2)

    jsActionLink = driver.find_elements(By.XPATH, '//a[@href="javascript:toggleDivVisibility(\'standard\');"]')
    if len(jsActionLink) > 0:
        jsActionLink[0].click()
        return(driver.page_source)

    return None

In [ ]:
def processTranslators(translatorList):
    for translator in translatorList:
        trltrtName = translator.get_text().strip().encode('utf-8').decode('utf-8')
        print(trltrtName)
        trltrPage = webprefix + translator.attrs['href']
        grabCV(trltrPage, trltrtName)
        time.sleep(3)    

In [ ]:
def openSoupify(url, s):
    r = requests.get(url, cookies=s.cookies)
    print("Status code: ", r.status_code)

    bs = BeautifulSoup(r.content, 'html.parser')
    translatorList = bs.findAll('a', {'class':'result-name profile-view'})
    processTranslators(translatorList)

    pagULs = bs.find('ul', {'class':'pagination'})
    pageNext = webprefix + pagULs.findAll('a', href=True, text=re.compile('Next'))[0].attrs['href']
    
    return pageNext

## Session handling

Log in and request the cookies programatically.

In [ ]:
session = requests.Session()

In [ ]:
params = {'username': username, 'password': password}

s = session.post(startpage, params)

cookieList = []
cookieList = [{"name" : key, "value"  : val} for key, val in s.cookies.get_dict().items()]

## Dynamic scraping initialization

In [ ]:
driver = webdriver.Firefox()

## Get and analize start page

In [ ]:
nextPage = openSoupify(startpage, s)

## Iteration through the complete result set

In [ ]:
while (nextPage != ""):
    print(nextPage)
    nextPage = openSoupify(nextPage, s)

ToDo:
- Problem with UTF8 translator names
- Big delay When it does not find the CV tag in Selenium
- while (nextPage is not None) is not a valid check
- Does not grab the CV in certain pages (https://www.proz.com/profile/3745000?phpv_redirected=1)



In [ ]:
grabCV("https://www.proz.com/profile/3745000", "Litzy Rafael-Osorio")